In [85]:
from time import sleep
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output # необходимо для удобного вывода информации
import json
import pprint
import re
import mcfg
import requests
from sqlalchemy import create_engine

In [29]:
location = 'Москва' # населенный пункт
title = 'Почта' # заведение
count_of_units = 5000 # объем выборки

In [30]:
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)  # без картинок

In [31]:
driver = webdriver.Chrome(options=options)
# Открыть URL
url = f'https://yandex.ru/maps/213/moscow/search/{location}%20{title}'
driver.get(url)

In [32]:
# Ожидание элемента до тех пор, пока он не будет найден
wait = WebDriverWait(driver, 10)  # 10 секунд ожидания
elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".search-snippet-view .link-overlay")))

In [33]:
n = 0
while len(elements)<count_of_units:
    clear_output(wait=True) # удаления предыдущего print и замены его на новый
    sleep(1)
    elements1 = len(elements)
    elements = driver.find_elements(By.CLASS_NAME, "search-business-snippet-view__content")
    driver.execute_script("arguments[0].scrollIntoView(true);", elements[-1])
    elements = driver.find_elements(By.CLASS_NAME, "search-business-snippet-view__content")
    elements2 = len(elements)
    print(f'кол-во - {len(elements)}', end=' ')
    # Дополнительный, цикл, если список больше не обновляется, через 30 попыток, цикл отключается
    if elements1 == elements2:
        n = n+1
        print('n = ', n, end=' ')
        if n >=10:
            break
    else:
        n = 0

кол-во - 805 n =  10 

In [36]:
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b2caf9b9", element="f.245BFBC2396AF9493DC64C65618721D0.d.45D455FDBFB5FEA68E61D9786B93ED7B.e.72")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b2caf9b9", element="f.245BFBC2396AF9493DC64C65618721D0.d.45D455FDBFB5FEA68E61D9786B93ED7B.e.73")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b2caf9b9", element="f.245BFBC2396AF9493DC64C65618721D0.d.45D455FDBFB5FEA68E61D9786B93ED7B.e.74")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b2caf9b9", element="f.245BFBC2396AF9493DC64C65618721D0.d.45D455FDBFB5FEA68E61D9786B93ED7B.e.75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b2caf9b9", element="f.245BFBC2396AF9493DC64C65618721D0.d.45D455FDBFB5FEA68E61D9786B93ED7B.e.76")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa61e29d0601184585e798f1b

In [47]:
for el in elements:
    pprint.pprint(el.get_attribute("outerHTML"))
    

('<div class="search-business-snippet-view__content"><div aria-hidden="true" '
 'class="search-business-snippet-view__photo"><span class="a11y-hidden"><a '
 'class="link-wrapper" '
 'href="/maps/org/otdeleniye_pochtovoy_svyazi_115191/1149438260/gallery/" '
 'tabindex="-1" aria-hidden="true">Фото</a></span><div '
 'class="photos-thumbnail-view _type_serp"><img class="img-with-alt" '
 'width="100%" height="100%" '
 'src="https://avatars.mds.yandex.net/get-altay/1003740/2a000001850bc4003a411400f1823f1977b9/S_height" '
 'alt="Отделение почтовой связи № 115191 (Большая Тульская ул., 2, Москва), '
 'почтовое отделение в Москве" style="object-fit: contain;"></div></div><div '
 'class="search-business-snippet-view__head"><div '
 'class="search-business-snippet-view__title">Отделение почтовой связи № '
 '115191<span class="business-verified-badge _prioritized" '
 'aria-hidden="true"><span class="inline-image _loaded icon" '
 'aria-hidden="true" role="button" tabindex="-1" style="font-size: 0px;

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [68]:
mail_df = pd.DataFrame(columns=('id', 'name', 'type', 'address'))
k = []
count = 0
for el in elements:
    html_snippet = el.get_attribute("outerHTML")
    soup = BeautifulSoup(html_snippet, "html.parser")
    img_tag = soup.find('img')
    if img_tag:
        count+=1
        alt_text = img_tag.get('alt') 
        
        start_paren = alt_text.find('(')
        end_paren = alt_text.find(')')
    
        name_part = alt_text[:start_paren].strip()
        address_part = alt_text[start_paren+1:end_paren].strip()
        
        # Считаем, что после закрывающей скобки идёт запятая и что-то ещё
        rest_part = alt_text[end_paren+1:].strip()
        k.append([name_part, address_part, rest_part])

        mail_df.loc[len(mail_df)] = [count, name_part, rest_part[2:], address_part]
        #df['id'] = count
        #df['name'] = name_part
        #df['type'] = rest_part
        #df['address'] = address_part
mail_df.to_excel('mail.xlsx')

PermissionError: [Errno 13] Permission denied: 'mail.xlsx'

In [69]:
mail_df

,id,name,type,address
0,1,Отделение почтовой связи № 115191,почтовое отделение в Москве,"Большая Тульская ул., 2, Москва"
1,2,Отделение почтовой связи № 115280,почтовое отделение в Москве,"Автозаводская ул., 5, Москва"
2,3,Отделение почтовой связи № 115455,почтовое отделение в Москве,"Валовая ул., 11/19, Москва"
3,4,Отделение почтовой связи № 115054,почтовое отделение в Москве,"Дубининская ул., 11, стр. 2, Москва"
4,5,Отделение почтовой связи № 115093,почтовое отделение в Москве,"Большая Серпуховская ул., 40, стр. 1, Москва"
...,...,...,...,...
788,789,Почтовое отделение № 141030,почтовое отделение в Мытищах,"1-я Пролетарская ул., 12, Мытищи"
789,790,Почта России Цех перечисления переводов,денежные переводы в Москве,"1-я Тверская-Ямская ул., 6, Москва"
790,791,Почтомат Pulse express,почтовое отделение в Москве,"Петровско-Разумовский пр., 2, Москва"
791,792,Почта России,постамат в Москве,"Нижегородская ул., 72/2, Москва"


In [70]:
mail_df = mail_df[mail_df['type'].str.startswith("почтовое отделение")]

In [71]:
mail_df

,id,name,type,address
0,1,Отделение почтовой связи № 115191,почтовое отделение в Москве,"Большая Тульская ул., 2, Москва"
1,2,Отделение почтовой связи № 115280,почтовое отделение в Москве,"Автозаводская ул., 5, Москва"
2,3,Отделение почтовой связи № 115455,почтовое отделение в Москве,"Валовая ул., 11/19, Москва"
3,4,Отделение почтовой связи № 115054,почтовое отделение в Москве,"Дубининская ул., 11, стр. 2, Москва"
4,5,Отделение почтовой связи № 115093,почтовое отделение в Москве,"Большая Серпуховская ул., 40, стр. 1, Москва"
...,...,...,...,...
783,784,Пункт приема и отправки посылок,почтовое отделение в Котельниках,"Котельники, микрорайон Белая Дача, 6"
785,786,Почта,почтовое отделение в Москве,"Туристская ул., 20, корп. 1, Москва"
788,789,Почтовое отделение № 141030,почтовое отделение в Мытищах,"1-я Пролетарская ул., 12, Мытищи"
790,791,Почтомат Pulse express,почтовое отделение в Москве,"Петровско-Разумовский пр., 2, Москва"


In [77]:
counter = 0
for i in list(mail_df['address']):
    counter +=1
    url = url = f"https://geocode-maps.yandex.ru/1.x/?apikey={mcfg.api_key()}&geocode={i}&format=json"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        geo_object = data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]
        coordinates = geo_object["Point"]["pos"]
        mail_df.loc[mail_df['address'] == i, 'post_coords'] = coordinates
    print(counter, str(response.status_code))

1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 200
20 200
21 200
22 200
23 200
24 200
25 200
26 200
27 200
28 200
29 200
30 200
31 200
32 200
33 200
34 200
35 200
36 200
37 200
38 200
39 200
40 200
41 200
42 200
43 200
44 200
45 200
46 200
47 200
48 200
49 200
50 200
51 200
52 200
53 200
54 200
55 200
56 200
57 200
58 200
59 200
60 200
61 200
62 200
63 200
64 200
65 200
66 200
67 200
68 200
69 200
70 200
71 200
72 200
73 200
74 200
75 200
76 200
77 200
78 200
79 200
80 200
81 200
82 200
83 200
84 200
85 200
86 200
87 200
88 200
89 200
90 200
91 200
92 200
93 200
94 200
95 200
96 200
97 200
98 200
99 200
100 200
101 200
102 200
103 200
104 200
105 200
106 200
107 200
108 200
109 200
110 200
111 200
112 200
113 200
114 200
115 200
116 200
117 200
118 200
119 200
120 200
121 200
122 200
123 200
124 200
125 200
126 200
127 200
128 200
129 200
130 200
131 200
132 200
133 200
134 200
135 200
136 200
137 200
138 200
139 

In [78]:
mail_df

,id,name,type,address,post_coords
0,1,Отделение почтовой связи № 115191,почтовое отделение в Москве,"Большая Тульская ул., 2, Москва",37.62077 55.709018
1,2,Отделение почтовой связи № 115280,почтовое отделение в Москве,"Автозаводская ул., 5, Москва",37.662084 55.70846
2,3,Отделение почтовой связи № 115455,почтовое отделение в Москве,"Валовая ул., 11/19, Москва",37.632368 55.730228
3,4,Отделение почтовой связи № 115054,почтовое отделение в Москве,"Дубининская ул., 11, стр. 2, Москва",37.638431 55.728023
4,5,Отделение почтовой связи № 115093,почтовое отделение в Москве,"Большая Серпуховская ул., 40, стр. 1, Москва",37.626582 55.723283
...,...,...,...,...,...
783,784,Пункт приема и отправки посылок,почтовое отделение в Котельниках,"Котельники, микрорайон Белая Дача, 6",37.847173 55.662146
785,786,Почта,почтовое отделение в Москве,"Туристская ул., 20, корп. 1, Москва",37.421731 55.85678
788,789,Почтовое отделение № 141030,почтовое отделение в Мытищах,"1-я Пролетарская ул., 12, Мытищи",37.769316 55.923009
790,791,Почтомат Pulse express,почтовое отделение в Москве,"Петровско-Разумовский пр., 2, Москва",37.57033 55.792154


In [ ]:
longitude 37

In [ ]:
latitude 55

In [82]:
mail_df[['latitude', 'longitude']] = (
    mail_df['post_coords']
    .str.split(' ', expand=True)
    .astype(float)
)

C:\Users\User\AppData\Local\Temp\ipykernel_21324\2446504873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail_df[['latitude', 'longitude']] = (
C:\Users\User\AppData\Local\Temp\ipykernel_21324\2446504873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail_df[['latitude', 'longitude']] = (


In [83]:
mail_df

,id,name,type,address,post_coords,latitude,longitude
0,1,Отделение почтовой связи № 115191,почтовое отделение в Москве,"Большая Тульская ул., 2, Москва",37.62077 55.709018,37.620770,55.709018
1,2,Отделение почтовой связи № 115280,почтовое отделение в Москве,"Автозаводская ул., 5, Москва",37.662084 55.70846,37.662084,55.708460
2,3,Отделение почтовой связи № 115455,почтовое отделение в Москве,"Валовая ул., 11/19, Москва",37.632368 55.730228,37.632368,55.730228
3,4,Отделение почтовой связи № 115054,почтовое отделение в Москве,"Дубининская ул., 11, стр. 2, Москва",37.638431 55.728023,37.638431,55.728023
4,5,Отделение почтовой связи № 115093,почтовое отделение в Москве,"Большая Серпуховская ул., 40, стр. 1, Москва",37.626582 55.723283,37.626582,55.723283
...,...,...,...,...,...,...,...
783,784,Пункт приема и отправки посылок,почтовое отделение в Котельниках,"Котельники, микрорайон Белая Дача, 6",37.847173 55.662146,37.847173,55.662146
785,786,Почта,почтовое отделение в Москве,"Туристская ул., 20, корп. 1, Москва",37.421731 55.85678,37.421731,55.856780
788,789,Почтовое отделение № 141030,почтовое отделение в Мытищах,"1-я Пролетарская ул., 12, Мытищи",37.769316 55.923009,37.769316,55.923009
790,791,Почтомат Pulse express,почтовое отделение в Москве,"Петровско-Разумовский пр., 2, Москва",37.57033 55.792154,37.570330,55.792154


In [86]:
# Параметры подключения к БД
# Создаем подключение и сразу закрываем его

db_host = "51.250.99.11"
db_port = "5432"
db_name = "postgres"
db_user = "team18"
schema_name = 'team18'
db_password = "team_18"
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
engine.dispose()

In [87]:
mail_df.to_sql('post_office', engine, schema=schema_name, if_exists='replace', index=False) # Загружаем датафрейм в БД

763